In [54]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.inspection import permutation_importance
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import GridSearchCV


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree



In [29]:
data = pd.read_csv('features.csv')
# Delete unesessary features from dataset
data = data.dropna()
data = data.drop(['enterworld_num','buyitemnowmainauction_num','completechallengeweek_num'],axis=1)
data = data.drop(['actor_account_id'
                    ,'survival_time'
                    ],axis=1)
data = data.drop(columns=data.columns[0],axis=1)
# Setup & split dataset for training
X = data.copy()
y = X.pop('churn_yn')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)
# Print training data shape
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)
# Get feature names from training set
feature_names = [f"feature {i}" for i in range(X.shape[1])]

(995, 33) (995,)
(2985, 33) (2985,)


In [44]:
X = data.copy()
y = X.pop('churn_yn')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)

In [41]:
clf = DecisionTreeClassifier(criterion='entropy', 
    splitter='best', 
    max_depth=6, 
    min_samples_split=2, 
    min_samples_leaf=4, 
    min_weight_fraction_leaf=0.0, 
    max_features='sqrt', 
    random_state=None, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    ccp_alpha=0.0,
    class_weight={0:1, 1:1 , 2:0.9 , 3:0.2, 4:0.2, 5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0}
)

In [42]:
forest = RandomForestClassifier(random_state=0,
                                criterion='entropy',
                                max_depth=6,
                                max_features='sqrt',
                                n_estimators=1000,
                                class_weight={0:1, 1:1 , 2:0.9 , 3:0.2, 4:0.2, 5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0})


In [64]:
knear= KNeighborsClassifier(3),

In [88]:
gapc=GaussianProcessClassifier(1.0 * RBF(1.0), multi_class=('one_vs_rest'),max_iter_predict=100, n_restarts_optimizer=3,n_jobs=-1)

In [89]:
eclf = VotingClassifier(estimators=[('RF', forest),
                                    ('Gaussian Process', gapc),
                                    #('Netural Ne', knear),
                                    ('DCTree', clf)],
                                    voting='soft', weights=[1,1,1])

eclf.fit(X_train, y_train)

In [79]:
filename = 'models/voting_clf.sav'
pickle.dump(eclf, open(filename, 'wb'))

In [80]:
#F1-score
print("F1 Score")
vote = cross_val_score(eclf, X, y, scoring="f1", cv = 5)
print("CrossValidation Scores: ")
print(vote)
print("%0.2f accuracy with a standard deviation of %0.2f" % (vote.mean(), vote.std()))

F1 Score


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a be

CrossValidation Scores: 
[0.56038647 0.50117096 0.56930693 0.45502646 0.57013575]
0.53 accuracy with a standard deviation of 0.05
